In [ ]:
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей,
которые выставили более 100 оценок. Под временем жизни понимается разница между 
максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
import pandas as pd

In [22]:
ratings = pd.read_csv('ratings.csv')

In [23]:
rat_count = ratings.groupby(['userId']).count().rename(columns={'rating': 'counts'})
rat_count = rat_count.drop(['movieId','timestamp'], axis='columns').reset_index()

In [5]:
time_max = ratings.groupby(['userId']).max().drop(['movieId','rating'], axis='columns').reset_index().rename(columns={'userId': 'userId_max'})
time_max.head()

,userId_max,timestamp
0,1,1260759205
1,2,835356246
2,3,1298932787
3,4,949982274
4,5,1163375145


In [6]:
time_min = ratings.groupby(['userId']).min().drop(['movieId','rating'], axis='columns').reset_index()
time_min = time_min.rename(columns={'timestamp': 'timestamp_min'}).rename(columns={'userId': 'userId_min'})
time_min.head()

,userId_min,timestamp_min
0,1,1260759108
1,2,835355395
2,3,1298861589
3,4,949778714
4,5,1163373044


In [7]:
cm = rat_count.join(time_max, how='left').rename(columns={'userId': 'user'}).reset_index()
all_data = cm.join(time_min, how='left')
all_data.head()

,index,user,counts,userId_max,timestamp,userId_min,timestamp_min
0,0,1,20,1,1260759205,1,1260759108
1,1,2,76,2,835356246,2,835355395
2,2,3,51,3,1298932787,3,1298861589
3,3,4,204,4,949982274,4,949778714
4,4,5,100,5,1163375145,5,1163373044


In [21]:
def time1(x):
    if x['counts'] >= 100:
        y = x['timestamp'] - x['timestamp_min']
    else:
        y = 0
    return y
all_data['time'] = all_data.apply(time1 , axis = 1)
all_data[all_data.counts > 100].time.mean()


40080507.4496124

In [ ]:

Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [26]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [27]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [28]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [29]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [36]:
rzd_auto = auto.merge(rzd, how='outer')
rzd_auto_air = rzd_auto.merge(air, how='outer')
rzd_auto_air.head(10)

,client_id,auto_revenue,rzd_revenue,air_revenue
0,113,57483.0,10283.0,NaN
1,114,83.0,5774.0,NaN
2,115,912.0,981.0,81.0
3,116,4834.0,NaN,4.0
4,117,98.0,NaN,13.0
5,111,NaN,1093.0,NaN
6,112,NaN,2810.0,NaN
7,118,NaN,NaN,173.0


In [37]:
data = client_base.merge(rzd_auto_air, how='outer')
data.head(10)

,client_id,address,auto_revenue,rzd_revenue,air_revenue
0,111,Комсомольская 4,NaN,1093.0,NaN
1,112,Энтузиастов 8а,NaN,2810.0,NaN
2,113,Левобережная 1а,57483.0,10283.0,NaN
3,114,Мира 14,83.0,5774.0,NaN
4,115,ЗЖБИиДК 1,912.0,981.0,81.0
5,116,Строителей 18,4834.0,NaN,4.0
6,117,Панфиловская 33,98.0,NaN,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


In [ ]:
Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. 
Т. е. для каждого user_id известна последовательность координат (широта/долгота), 
когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? 
Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат.
А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга.
Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [ ]:
Сведения можно добавить по user_id и времени с помощью команды merge - outer  для каждого визита или покупки. 
Если данные о местоположении выгружаются, как обычные данные геолокации, то их нужно будет разбить по столбцам 
на ширину-долготу- время метки с помощью regex. 
по координатам можно понять частые места посещения пользователя( дом-работа)
- можно предложить близлежайщие оффлайн магазины или пункты выдачи. По времени покупок можно попробовать найти наиболее активный
период пользования приложения.